# Inverse Control for pushing
### $V \rightarrow(F,f)\rightarrow p, v_p$
Find the **contact point**, and the **contact velocity** to achieve the desired twist of the object.

### Equations:
- $\tau = p_xF_y - p_yF_x$ (torque constraint)
- $(p_x,p_y)\in X(o)$ (shape constraint)
- $f^p_p\in$ cone($f_l,f_r$) (motion cone)
- $f^p_p\cdot\hat{n}\geq0$, $v_p^p\cdot\hat{n}\geq0$ (impact should be toward obj)

## Fully-actuated
for any $V$, there exists a solution.

## Under-actuated
for some $V$, there may not exist solutions because of
- motion cone constraints
- direction of force/velocity
- line not intersecting with the shape
In these cases, we might need to find intermediate waypoints, i.e. we might need planning.

In [1]:
from refined_push_sim import *
from slider_geometry import *
from motion_cone import *
from slider_obj import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
import matplotlib as mpl
from IPython import display

from refined_point_push_animation import FreePointMoveSimulator
import utilities as utils

In [2]:
def inverse_control1(slider: Slider, twist):
    """
    given a specific velocity to achieve, plan for the contact point and contact velocity 
    for the pusher
    Fully-actuated: find exact solution
    Under-actuated: find the closest solution
    
    Alternative:
    for each point on the surface, find the force such that the twist is the closest to the
    target
    """
    # * find load corresponding to the twist
    load = slider.ls_model.get_load(twist.reshape((1,-1)))[0]
    
    # * find on slider (px, py) such that F[2] = px F[1] - py F[0]
    pcd = slider.geometry.surface_pcd
    diff = pcd[:,0]*load[1] - pcd[:,1]*load[0] - load[2]
    diff = np.abs(diff)
    
    # here we use the load generated to check the motion cone
    threshold = 1e-2
    mask = (diff < threshold)
    pts = pcd[mask]
    diff = diff[mask]
#     idx = np.argmin(diff)
#     pt = pcd[idx]
    
    # * check constraints
    f = np.array([load[0],load[1]])
    v = np.zeros((len(pts),2))
    v[:,0] = twist[0] - pts[:,1]*twist[2]
    v[:,1] = twist[1] + pts[:,0]*twist[2]
#     v = np.array([twist[0] - pt[1]*twist[2], twist[1] + pt[0]*twist[2]])
    norm = slider.geometry.get_normal(pts)
    ang_max = slider.motion_cone.get_angle(pts)
    
    # making sure if f is inside motion cone
    ang = np.arccos(norm.dot(f)/np.linalg.norm(f))
    
#     if ang <= ang_max:
#         print('inside motion cone')
#     else:
#         print('outside motion cone')
    v_dot = v * norm
    v_dot = v_dot.sum(axis=1) / np.linalg.norm(v,axis=1)
    
    if (v_dot>=0).astype(int).sum() > 0:
        pts = pts[v_dot>=0,:]
        diff = diff[v_dot>=0]
        v = v[v_dot>=0,:]
        idx = np.argmin(diff)
        print('velocity correct')
        return pts[idx], v[idx]
    else:
        print('velocity incorrect')
        # use the force to generate a velocity
        idx = np.argmax(v_dot)
        pt = pts[idx]
        # check if motion within range
        ang = ang[idx]
        normal = self.geometry.get_normal(pt).reshape(-1)
        mc_angle = self.motion_cone.get_angle(pt)
        fl = utils.rot_mat_from_ang(-mc_angle).dot(normal)
        fu = utils.rot_mat_from_ang(mc_angle).dot(normal)
        
        if np.abs(ang) > ang_max:
            if ang > 0:
                f = fu
            else:
                f = fl
        else:
            f = np.array([load[0], load[1]])
        F = np.array([f[0], f[1], pt[0]*f[1]-pt[1]*f[0]])
        
        V = slider.ls_model.get_twist(F)
        V = V * np.linalg.norm(twist) / np.linalg.norm(V)  # keep same length
        v = np.zeros((2))
        v[0] = V[0] - V[2]*pt[1]
        v[1] = V[1] + V[2]*pt[0]
        return pt, v


In [3]:
def inverse_control(slider: Slider, twist):
    """
    """
    # * find load corresponding to the twist
    load = slider.ls_model.get_load(twist.reshape((1,-1)))[0]
    
    # * find on slider (px, py) such that F[2] = px F[1] - py F[0]
    pcd = slider.geometry.surface_pcd
    
    f = np.array([load[0], load[1]])
    
    normals = slider.geometry.get_normal(pcd)  # N x 2
    ang_max = slider.motion_cone.get_angle(pcd)
    fu = utils.rot_mat_from_ang(ang_max)
    fl = utils.rot_mat_from_ang(-ang_max)
    fu = fu.dot(normals.T).T
    fl = fl.dot(normals.T).T

    # edit: motion cone constraint
    angs = normals.dot(f) / np.linalg.norm(normals, axis=1) / np.linalg.norm(f)
    angs = np.arccos(angs)
    lower_mask = (angs < -ang_max)
    upper_mask = (angs > ang_max)
    F = np.zeros((len(normals),3))
    F[:,:2] = f
    F[lower_mask,:2] = fl[lower_mask, :2]
    F[upper_mask,:2] = fu[upper_mask, :2]
    
    F[:,2] = pcd[:,0]*F[:,1] - pcd[:,1]*F[:,0]
    
    V = slider.ls_model.get_twist(F) # get closest to twist
    
    V = V / np.linalg.norm(V, axis=1, keepdims=True)
    idx = np.argmax(V.dot(twist))
    V = V[idx] * np.linalg.norm(twist)
    pt = pcd[idx]
    v = np.zeros((2))
    v[0] = V[0] - V[2]*pt[1]
    v[1] = V[1] + V[2]*pt[0]
    return pt, v


In [4]:
# * Rectangular shape
width = 0.8
height = 0.4
pcd = np.random.uniform(low=(-width/2, -height/2), high=(width/2, height/2),
                        size=(4000,2))
resols = np.array([0.05, 0.05])
pcd, grid = utils.pcd_to_grid(pcd, resols)

friction_coeff = np.zeros(grid.shape)
pressure_dist = np.zeros(grid.shape)
friction_coeff[grid] = 1
pressure_dist[grid] = 1
geometry = GridGeometry(pcd, grid, resols)
grid_ls = GridLimitSurface(friction_coeff, pressure_dist, resols)
grid_ls.construct_limit_surface(n_samples=75000)
motion_cone = SimpleMotionCone(np.pi / 6)

slider = Slider(geometry, grid_ls, motion_cone, np.eye(3))

Fkdtree shape:  (75000, 3)
Vkdtree shape:  (75000, 3)


In [5]:
init_transform = np.zeros((3))
target_transform = np.array([0.2, 0.2, 30*np.pi/180])
dt = 0.01
vel_scale = 0.05

pt_tfs_in_slider = []
pt_vels_in_slider = []

transform = np.array(init_transform)
# get the pt and velocity lists
while True:
    diff = target_transform - transform
    diff[2] = (diff[2] + np.pi) % (2*np.pi) - np.pi
    abs_diff = np.abs(diff)
    if abs_diff[0] <= 1e-3 and abs_diff[1] <= 1e-3 and abs_diff[2] <= 1*np.pi/180:
        break
    twist = diff / np.linalg.norm(diff) * vel_scale
    pt, vel = inverse_control(slider, twist)
    F = slider.point_pusher_model(pt, vel)
    twist = slider.ls_model.get_twist(F.reshape((1,-1)))[0]
    # get the pt and vel in the world frame
    slider_in_world = utils.transform_vec_to_mat(transform)
#     pt_in_world = slider_in_world[:2,:2].dot(pt) + slider_in_world[:2,2]
#     vel_in_world = slider_in_world[:2,:2].dot(vel) 
    
    pt_tfs_in_slider.append(pt)
    pt_vels_in_slider.append(vel)
    
    transform = transform + twist/np.linalg.norm(twist)*vel_scale * dt
print('number of points: ', len(pt_tfs_in_slider))
print('final transform: ', transform)
print('target: ', target_transform)
# point_transform = np.array([0.3, -0.25])
# point_vel_list = np.zeros((800,2))
# point_vel_list[:,1] = 0.05
# point_vel_list = np.array([0.0,0.01])
sim = FreePointMoveSimulator(slider, init_transform, pt_tfs_in_slider, pt_vels_in_slider, dt)
sim.anim()

/home/yinglong/Documents/research/control/code/control_playground/scripts/pushing/slider_geometry.py:92: RuntimeWarning: invalid value encountered in divide
  grad = grad / np.linalg.norm(grad, axis=-1, keepdims=True)


number of points:  1185
final transform:  [0.19901549 0.19906889 0.52113629]
target:  [0.2        0.2        0.52359878]
V:  [0.01628619 0.01704954 0.04217375]
V:  [0.016279   0.01705641 0.04217375]
V:  [0.0162718  0.01706327 0.04217375]
V:  [0.01626461 0.01707013 0.04217375]
V:  [0.01625741 0.01707699 0.04217375]
V:  [0.0162502  0.01708385 0.04217375]
V:  [0.016243   0.0170907  0.04217375]
V:  [0.01623579 0.01709755 0.04217375]
V:  [0.01622857 0.01710439 0.04217375]
V:  [0.01622136 0.01711124 0.04217375]
V:  [0.01621414 0.01711807 0.04217375]
V:  [0.01620692 0.01712491 0.04217375]
V:  [0.0161997  0.01713174 0.04217375]
V:  [0.01619247 0.01713858 0.04217375]
V:  [0.01618524 0.0171454  0.04217375]
V:  [0.01617801 0.01715223 0.04217375]
V:  [0.01617077 0.01715905 0.04217375]
V:  [0.01616354 0.01716587 0.04217375]
V:  [0.0161563  0.01717268 0.04217375]
V:  [0.01614905 0.01717949 0.04217375]
V:  [0.0161418  0.0171863  0.04217375]
V:  [0.01613456 0.01719311 0.04217375]
V:  [0.0161273  0.017

V:  [0.01474817 0.01841957 0.04220644]
V:  [0.0147404  0.01842579 0.04220644]
V:  [0.01473262 0.01843201 0.04220644]
V:  [0.01472484 0.01843823 0.04220644]
V:  [0.01471706 0.01844444 0.04220644]
V:  [0.01470927 0.01845065 0.04220644]
V:  [0.01470148 0.01845686 0.04220644]
V:  [0.01469369 0.01846306 0.04220644]
V:  [0.0146859  0.01846926 0.04220644]
V:  [0.0146781  0.01847546 0.04220644]
V:  [0.0146703  0.01848165 0.04220644]
V:  [0.0146625  0.01848784 0.04220644]
V:  [0.01465469 0.01849403 0.04220644]
V:  [0.01464689 0.01850021 0.04220644]
V:  [0.01463908 0.01850639 0.04220644]
V:  [0.01463127 0.01851257 0.04220644]
V:  [0.01462345 0.01851874 0.04220644]
V:  [0.01461563 0.01852491 0.04220644]
V:  [0.01460781 0.01853108 0.04220644]
V:  [0.01459999 0.01853724 0.04220644]
V:  [0.01459217 0.0185434  0.04220644]
V:  [0.01458434 0.01854956 0.04220644]
V:  [0.01457651 0.01855572 0.04220644]
V:  [0.01456867 0.01856187 0.04220644]
V:  [0.01456084 0.01856801 0.04220644]
V:  [0.014553   0.0185741

V:  [0.01298649 0.01963861 0.0421129 ]
V:  [0.01297821 0.01964408 0.0421129 ]
V:  [0.01296994 0.01964954 0.0421129 ]
V:  [0.01296256 0.01965636 0.04211582]
V:  [0.01295339 0.01966046 0.0421129 ]
V:  [0.0129451  0.01966591 0.0421129 ]
V:  [0.01293682 0.01967136 0.0421129 ]
V:  [0.01292943 0.01967817 0.04211582]
V:  [0.01292114 0.01968362 0.04211582]
V:  [0.01291285 0.01968906 0.04211582]
V:  [0.01290456 0.01969449 0.04211582]
V:  [0.01289626 0.01969993 0.04211582]
V:  [0.01288797 0.01970536 0.04211582]
V:  [0.01287967 0.01971078 0.04211582]
V:  [0.01287136 0.0197162  0.04211582]
V:  [0.01286306 0.01972162 0.04211582]
V:  [0.01464699 0.01920686 0.04434499]
V:  [0.012846   0.01973274 0.04211582]
V:  [0.01463038 0.01921952 0.04434499]
V:  [0.01282894 0.01974384 0.04211582]
V:  [0.01461376 0.01923216 0.04434499]
V:  [0.01281186 0.01975492 0.04211582]
V:  [0.01459713 0.01924479 0.04434499]
V:  [0.01279478 0.01976599 0.04211582]
V:  [0.01278645 0.01977138 0.04211582]
V:  [0.01457237 0.0192635

V:  [0.01079486 0.01991031 0.04040486]
V:  [0.01279976 0.02004176 0.04692339]
V:  [0.01077747 0.01991973 0.04040486]
V:  [0.01278225 0.02005293 0.04692339]
V:  [0.01076007 0.01992914 0.04040486]
V:  [0.01276473 0.02006409 0.04692339]
V:  [0.01074266 0.01993853 0.04040486]
V:  [0.01274721 0.02007523 0.04692339]
V:  [0.01072524 0.0199479  0.04040486]
V:  [0.01272967 0.02008635 0.04692339]
V:  [0.01070782 0.01995726 0.04040486]
V:  [0.01271213 0.02009746 0.04692339]
V:  [0.01069039 0.0199666  0.04040486]
V:  [0.01269457 0.02010855 0.04692339]
V:  [0.01067295 0.01997593 0.04040486]
V:  [0.012677   0.02011963 0.04692339]
V:  [0.0106555  0.01998524 0.04040486]
V:  [0.01265943 0.0201307  0.04692339]
V:  [0.01063804 0.01999454 0.04040486]
V:  [0.01264184 0.02014174 0.04692339]
V:  [0.01062058 0.02000382 0.04040486]
V:  [0.01262425 0.02015278 0.04692339]
V:  [0.0106031  0.02001309 0.04040486]
V:  [0.01260665 0.02016379 0.04692339]
V:  [0.01058562 0.02002234 0.04040486]
V:  [0.01258903 0.0201747

V:  [0.0090023  0.02096461 0.04070334]
V:  [0.01071848 0.02042825 0.04589008]
V:  [0.00898414 0.0209724  0.04070334]
V:  [0.01070079 0.02043752 0.04589008]
V:  [0.01069141 0.02044243 0.04589008]
V:  [0.00895634 0.02098428 0.04070334]
V:  [0.0106737  0.02045168 0.04589008]
V:  [0.00893817 0.02099203 0.04070334]
V:  [0.01065599 0.02046091 0.04589008]
V:  [0.00891999 0.02099976 0.04070334]
V:  [0.01063827 0.02047013 0.04589008]
V:  [0.01062887 0.02047501 0.04589008]
V:  [0.00889216 0.02101156 0.04070334]
V:  [0.01061114 0.02048421 0.04589008]
V:  [0.00887396 0.02101925 0.04070334]
V:  [0.0105934  0.02049339 0.04589008]
V:  [0.01058399 0.02049825 0.04589008]
V:  [0.00884611 0.02103099 0.04070334]
V:  [0.01056624 0.02050741 0.04589008]
V:  [0.00882789 0.02103864 0.04070334]
V:  [0.01054848 0.02051655 0.04589008]
V:  [0.00880967 0.02104628 0.04070334]
V:  [0.01053071 0.02052568 0.04589008]
V:  [0.01052128 0.02053051 0.04589008]
V:  [0.00878178 0.02105793 0.04070334]
V:  [0.0105035  0.0205396

V:  [0.00703043 0.02138451 0.04537126]
V:  [0.00699185 0.02171797 0.04070334]
V:  [0.00875086 0.02134529 0.04589008]
V:  [0.00874106 0.02134931 0.04589008]
V:  [0.00699238 0.02139698 0.04537126]
V:  [0.00695321 0.02173037 0.04070334]
V:  [0.00871288 0.02136082 0.04589008]
V:  [0.00870308 0.02136482 0.04589008]
V:  [0.00647125 0.02173698 0.04432877]
V:  [0.0086838  0.02137266 0.04589008]
V:  [0.0069048  0.0217458  0.04070334]
V:  [0.00866529 0.02138018 0.04589008]
V:  [0.00865547 0.02138415 0.04589008]
V:  [0.00642282 0.02175134 0.04432877]
V:  [0.00686634 0.02175798 0.04070334]
V:  [0.00862747 0.02139546 0.04589008]
V:  [0.00861765 0.02139942 0.04589008]
V:  [0.00683751 0.02176705 0.04070334]
V:  [0.00859912 0.02140687 0.04589008]
V:  [0.00858929 0.02141082 0.04589008]
V:  [0.00684032 0.02144608 0.04537126]
V:  [0.00679879 0.02177918 0.04070334]
V:  [0.00856103 0.02142213 0.04589008]
V:  [0.0085512  0.02142606 0.04589008]
V:  [0.00680217 0.02145821 0.04537126]
V:  [0.00676004 0.0217912